In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [1]:
!pip install --upgrade transformers datasets peft trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
# Importing packages
import os
import gc
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import DPOTrainer, DPOConfig
import bitsandbytes as bnb
from tqdm import tqdm

In [3]:
from datasets import load_dataset
dataset = load_dataset("yaditi1303/mental_health_preferred_dataset")['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def format_instruction(sample):
  sample['prompt'] = f"Question: {sample['prompt']}\nAnswer: "
  return sample

In [5]:
dataset = dataset.map(format_instruction)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [1]:
# Define model names and tokens
peft_model_name = "microsoft/Phi-3-mini-4k-instruct"
new_model = "MindPhi"

In [5]:
# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(peft_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [8]:
# LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj"]
)

In [9]:
# Load the base model with BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [10]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:
# Loads model from hugging face and device mapping
model = AutoModelForCausalLM.from_pretrained(
    peft_model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False

#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [12]:
# Training arguments
training_args = DPOConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=25,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=5,
)

In [13]:
# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:389: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:402: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:442: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
gc.collect()
torch.cuda.empty_cache()

In [15]:
# Fine-tune model with DPO
dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.690800
4,0.685600
5,0.677200
6,0.666000
7,0.653900
8,0.635700
9,0.623300
10,0.603700


TrainOutput(global_step=25, training_loss=0.5750317084789276, metrics={'train_runtime': 382.8648, 'train_samples_per_second': 0.522, 'train_steps_per_second': 0.065, 'total_flos': 0.0, 'train_loss': 0.5750317084789276, 'epoch': 20.0})

In [16]:
# Save artifacts
dpo_trainer.model.save_pretrained("dpo_model")
tokenizer.save_pretrained("dpo_model")

('dpo_model/tokenizer_config.json',
 'dpo_model/special_tokens_map.json',
 'dpo_model/tokenizer.model',
 'dpo_model/added_tokens.json',
 'dpo_model/tokenizer.json')

In [17]:
# Flush memory
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

In [6]:
# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_name)

In [8]:
# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "dpo_model")
model = model.merge_and_unload()

In [ ]:
model.push_to_hub(new_model, check_pr=True)
tokenizer.push_to_hub(new_model,check_pr=True)

In [ ]:
# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

In [9]:
model.to("cuda")

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
  )
 

In [52]:
import pandas as pd
df = pd.DataFrame(dataset)

In [53]:
responses=[]
for i in tqdm(range(len(df))):
    instruction = df['prompt'].values[i]
    prompt = "[INST] " + instruction +  " [/INST]"
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens = 100, temperature=0.9, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response.split('[/INST] ')[-1])

100%|██████████| 10/10 [00:47<00:00,  4.73s/it]


In [54]:
df['response'] = responses

In [55]:
df.values[1]

array(["Can I talk to someone if I'm feeling overwhelmed?",
       "Absolutely, talking to someone when you're feeling overwhelmed is very important. There are many support options available, such as friends, family, or mental health professionals. Would you like recommendations for support groups or counselors?",
       "While it's crucial to seek support, sometimes finding the right person to talk to can take time. If you're having trouble finding someone to talk to, I can help provide some resources.",
       "I'm committed to providing emotional support as a chatbot, and while I'm here to help with day-to-day concerns, feelings of being overwhelmed can be significant and complex. It's crucial to seek support from qualified human professionals, such as counselors, therapists, or mental health hotlines, for personalized care and assistance. They can provide a safe and confidential space to explore your feelings and develop strategies to cope"],
      dtype=object)

In [56]:
df

,prompt,chosen,rejected,response
0,Is it normal to feel anxious in social situati...,"Yes, it is quite common to feel anxious in soc...","While occasional anxiety is normal, if your an...","Yes, it is normal to experience anxiety in soc..."
1,Can I talk to someone if I'm feeling overwhelmed?,"Absolutely, talking to someone when you're fee...","While it's crucial to seek support, sometimes ...",I'm committed to providing emotional support a...
2,Are there any techniques to help manage stress?,"Yes, there are several techniques to manage st...","Unfortunately, not all techniques work for eve...",Stress is a natural response to the demands of...
3,Is it okay to take a mental health day from work?,"Yes, taking a mental health day is a great way...","While it can be beneficial, some workplaces mi...","Yes, it's generally considered acceptable to t..."
4,Can lifestyle changes improve my mental health?,"Yes, lifestyle changes such as regular exercis...","While lifestyle changes can help, they might n...","Yes, lifestyle changes can have a significant ..."
5,Are there specific signs that indicate someone...,"Yes, there are several signs such as persisten...","While there are common signs, they can vary gr...","Yes, there are specific signs that can indicat..."
6,Is therapy effective for treating mental healt...,"Yes, therapy has been proven to be very effect...","While therapy is effective for many, it might ...","Yes, therapy can be an effective treatment for..."
7,Can medication help with mental health conditi...,"Yes, medication can be very helpful for certai...",Medication might not be suitable for everyone ...,[INST] Can medication help with mental health ...
8,Are there online resources available for ment...,"Yes, there are many online resources availabl...","While online resources can be helpful, they mi...","Yes, there are numerous online resources avail..."
9,Can mindfulness meditation improve mental health?,"Yes, mindfulness meditation has been shown to ...",While mindfulness meditation can be beneficial...,Mindfulness meditation has been widely studied...


In [60]:
df.values[0]

array(['Yes, it is normal to experience anxiety in social situations. This feeling can range from a little nervousness to more intense forms of anxiety. Social anxiety disorder (SAD), formerly known as social phobia, is a chronic mental health condition characterized by significant anxiety and discomfort about being embarrassed, rejected, or looked down upon in social interactions.\n\nMost people feel a certain amount of nervousness when they are in new'],
      dtype=object)

In [58]:
df = pd.DataFrame({'Response': responses})


In [59]:
df

,Response
0,"Yes, it is normal to experience anxiety in soc..."
1,I'm committed to providing emotional support a...
2,Stress is a natural response to the demands of...
3,"Yes, it's generally considered acceptable to t..."
4,"Yes, lifestyle changes can have a significant ..."
5,"Yes, there are specific signs that can indicat..."
6,"Yes, therapy can be an effective treatment for..."
7,[INST] Can medication help with mental health ...
8,"Yes, there are numerous online resources avail..."
9,Mindfulness meditation has been widely studied...
